In [1]:
from bs4 import BeautifulSoup
import requests
import re
import json
from ast import literal_eval
import json
import urllib.request
import numpy as np
import pandas as pd
import time
import datetime
import random

In [2]:
end = False
solecollector_com = 'https://solecollector.com'
textify = lambda x: x.text
stripify = lambda x: x.strip()
sneaker_keys = ['id', 'name', 'date_created', 'date_updated', 'alias', 'release_date', 'images', 
                'description', 'style_code', 'original_price', 'hero_image', 'available_at', 'url', 
                'release_day', 'release_month', 'hero_image_url', 'hero_image_id', 'release_date_pretty', 
                'original_price_pretty', 'hero_image_alt', 'canonical_url']

sneaker_json = []
brand_json = []
secondary_brand_json = []
silhouette_json = []

In [3]:
current_year = datetime.datetime.now().year
current_month = datetime.datetime.now().month

current_months = [str(x).zfill(2) for x in range(1, current_month+1)]
all_months = [str(x).zfill(2) for x in range(1, 13)]
all_years = [str(x) for x in range(2011, current_year+1)]
dates_dict = {year:all_months for year in all_years}
dates_dict[str(current_year)] = current_months

urls = []
for year, months in dates_dict.items():
    for month in months:
        urls.append(f'https://solecollector.com/sneaker-release-dates/all-release-dates/{year}/{month}/')

In [5]:
def get_data(sneaker_soup, sneaker_script):
    
    # Obtain data on brand, secondary brand, and silhouette
    no_additional_data = False
    query1 = re.search('var sneaker_path = (.+)[,;]{1}', sneaker_script)
    if query1:
        query1_data = json.loads(query1.group(1))
        
        brand = query1_data[0]
        secondary_brand = query1_data[1]
        silhouette = query1_data[2]
        
        brand_id = brand['id']
        secondary_brand_id = secondary_brand['id']
        silhouette_id = silhouette['id']
        
    else:
        
        no_additional_data = True
        brand_id = np.nan
        secondary_brand_id = np.nan
        silhouette_id = np.nan
    
    # Obtain data on sneaker
    query2 = re.search('var sneaker_release = (.+)[,;]{1}', sneaker_script)
    if query2:
        sneaker = json.loads(query2.group(1))
        sneaker['brand_id'] = brand_id
        sneaker['secondary_brand_id'] = secondary_brand_id
        sneaker['silhouette_id'] = silhouette_id
        
    else:
        
        sneaker_values = sneaker_soup.find_all('div', class_='sd-brand-info__value')
        sneaker_values = list(map(textify, sneaker_values))
        sneaker_values = list(map(stripify, sneaker_values))
        
        sneaker = {key: np.nan for key in sneaker_keys} 
        sneaker['brand_id'] = brand_id
        sneaker['secondary_brand_id'] = secondary_brand_id
        sneaker['silhouette_id'] = silhouette_id
        sneaker['release_date_pretty'] = sneaker_values[0]
        sneaker['style_code'] = sneaker_values[1]
        sneaker['name'] = sneaker_values[2]
        sneaker['original_price_pretty'] = sneaker_values[3]
        
    if no_additional_data == False:
        sneaker_json.append(sneaker)
        brand_json.append(brand)
        secondary_brand_json.append(secondary_brand)
        silhouette_json.append(silhouette)
        
    else:
        sneaker_json.append(sneaker)

In [6]:
while urls:
    
    current_url = random.choice(urls)
    
    try:
        source = requests.get(current_url, time.sleep(3)).text
    except:
        continue
    
    source_soup = BeautifulSoup(source)
    monthly_sneakers = source_soup.find_all('div', class_='col-xs-12 col-sm-6')
    
    for index, sneaker in enumerate(monthly_sneakers):
        if index % 2:
            continue
            
        sneaker_url = solecollector_com + monthly_sneakers[index].a['href']
        sneaker_html = requests.get(sneaker_url).text
        sneaker_soup = BeautifulSoup(sneaker_html)
        
        try:
            sneaker_script = str(sneaker_soup.find_all('script', type="text/javascript")[2])
        except IndexError:
            break
        get_data(sneaker_soup, sneaker_script)
    
    urls.remove(current_url)
    print(len(urls))
    

113
112
111
110
109
108
107
106
105
104
103
102
101
100
99
98
97
96
95
94
93
92
91
90
89
88
87
86
85
84
83
82
81
80
79
78
77
76
75
74
73
72
71
70
69
68
67
66
65
64
63
62
61
60
59
58
57
56
55
54
53
52
51
50
49
48
47
46
45
44
43
42
41
40
39
38
37
36
35
34
33
32
31
30
29
28
27
26
25
24
23
22
21
20
19
18
17
16
15
14
13
12
11
10
9
8
7
6
5
4
3
2
1
0


In [17]:
with open('/Users/nikhilbhargava/Documents/sneakairs/data/01_raw/solecollector/sneaker.json', 'w') as outfile:
    json.dump(sneaker_json, outfile)

sneaker_df = pd.DataFrame(sneaker_json)
sneaker_df = sneaker_df.sort_values(by='release_date', ascending=False)
sneaker_df.to_csv('/Users/nikhilbhargava/Documents/sneakairs/data/01_raw/solecollector/sneaker.csv', index=False)

In [18]:
with open('/Users/nikhilbhargava/Documents/sneakairs/data/01_raw/solecollector/brand.json', 'w') as outfile:
    json.dump(brand_json, outfile)

brand_df = pd.DataFrame(brand_json)
brand_df = brand_df.sort_values(by='id', ascending=True)
brand_df.to_csv('/Users/nikhilbhargava/Documents/sneakairs/data/01_raw/solecollector/brand.csv', index=False)

In [19]:
with open('/Users/nikhilbhargava/Documents/sneakairs/data/01_raw/solecollector/secondary_brand.json', 'w') as outfile:
    json.dump(secondary_brand_json, outfile)

secondary_brand_df = pd.DataFrame(secondary_brand_json)
secondary_brand_df = secondary_brand_df.sort_values(by='id', ascending=True)
secondary_brand_df.to_csv('/Users/nikhilbhargava/Documents/sneakairs/data/01_raw/solecollector/secondary_brand.csv', index=False)

In [20]:
with open('/Users/nikhilbhargava/Documents/sneakairs/data/01_raw/solecollector/silhouette.json', 'w') as outfile:
    json.dump(silhouette_json, outfile)

silhouette_df = pd.DataFrame(silhouette_json)
silhouette_df = silhouette_df.sort_values(by='id', ascending=True)
silhouette_df.to_csv('/Users/nikhilbhargava/Documents/sneakairs/data/01_raw/solecollector/silhouette.csv', index=False)